# Day 10: Factory

## Problem Description

We need to configure machines by pressing buttons.
Each machine has:
1. An initial state of indicator lights (all off).
2. A target state of indicator lights.
3. A set of buttons, each toggling a subset of lights.
4. A set of joltage counters (initially 0).
5. A target state of joltage counters.
6. The same buttons, but now they increment the counters by 1.

**Part 1:** Find the minimum number of button presses to match the indicator lights. (GF(2) system)
**Part 2:** Find the minimum number of button presses to match the joltage counters. (Integer linear system)

## Input Parsing

We need to parse lines like:
`[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}`

- `[...]`: Indicator light target (binary string, `.`=0, `#`=1).
- `(...)`: Button schematics (list of indices affected).
- `{...}`: Joltage requirements (list of integers).

In [ ]:
import re

def parse_line(line):
    # Extract indicator lights
    lights_match = re.search(r'\[([.#]+)\]', line)
    lights_str = lights_match.group(1)
    lights = [1 if c == '#' else 0 for c in lights_str]
    
    # Extract buttons
    # Buttons are in (parentheses). There can be multiple.
    # We can find all occurrences of (...)
    buttons_matches = re.findall(r'\(([\d,]+)\)', line)
    buttons = []
    for b in buttons_matches:
        indices = [int(x) for x in b.split(',')]
        buttons.append(indices)
        
    # Extract joltage requirements
    joltage_match = re.search(r'\{([\d,]+)\}', line)
    joltage_str = joltage_match.group(1)
    joltage = [int(x) for x in joltage_str.split(',')]
    
    return {
        'lights': lights,
        'buttons': buttons,
        'joltage': joltage
    }

def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return [parse_line(line.strip()) for line in lines if line.strip()]

# Test with test.txt
test_machines = read_input('test.txt')
for m in test_machines:
    print(m)


In [ ]:
machines = read_input('input.txt')
print(f"Number of machines: {len(machines)}")
if len(machines) > 0:
    print(f"First machine keys: {machines[0].keys()}")

max_buttons = max(len(m['buttons']) for m in machines)
print(f"Max buttons: {max_buttons}")
max_lights = max(len(m['lights']) for m in machines)
print(f"Max lights: {max_lights}")


In [ ]:
from z3 import *

def solve_part1(machine):
    lights = machine['lights']
    buttons = machine['buttons']
    num_lights = len(lights)
    num_buttons = len(buttons)
    
    # Create binary variables for each button
    x = [Int(f'x_{i}') for i in range(num_buttons)]
    
    opt = Optimize()
    
    # Constraints: x_i must be 0 or 1
    for i in range(num_buttons):
        opt.add(x[i] >= 0)
        opt.add(x[i] <= 1)
        
    # Constraints for each light
    for j in range(num_lights):
        # Sum of x_i for buttons that affect light j
        # We want sum(x_i * b_ij) % 2 == target_j
        # In Z3 with Ints, we can say sum(...) = 2*k_j + target_j
        
        expr = Sum([x[i] for i in range(num_buttons) if j in buttons[i]])
        
        # Introduce an auxiliary integer variable for the multiple of 2
        k = Int(f'k_{j}')
        opt.add(expr == 2 * k + lights[j])
        
    # Minimize sum of x_i
    opt.minimize(Sum(x))
    
    if opt.check() == sat:
        model = opt.model()
        return sum(model[xi].as_long() for xi in x)
    else:
        print("No solution found for machine")
        return 0

def solve_part2(machine):
    joltage = machine['joltage']
    buttons = machine['buttons']
    num_counters = len(joltage)
    num_buttons = len(buttons)
    
    # Create integer variables for each button
    x = [Int(f'x_{i}') for i in range(num_buttons)]
    
    opt = Optimize()
    
    # Constraints: x_i >= 0
    for i in range(num_buttons):
        opt.add(x[i] >= 0)
        
    # Constraints for each counter
    for j in range(num_counters):
        # Sum of x_i for buttons that affect counter j
        # We want sum(x_i * b_ij) == target_j
        
        expr = Sum([x[i] for i in range(num_buttons) if j in buttons[i]])
        opt.add(expr == joltage[j])
        
    # Minimize sum of x_i
    opt.minimize(Sum(x))
    
    if opt.check() == sat:
        model = opt.model()
        return sum(model[xi].as_long() for xi in x)
    else:
        print("No solution found for machine (Part 2)")
        return 0

# Test on test data
print("Part 1 Test:")
total_p1 = 0
for m in test_machines:
    res = solve_part1(m)
    print(res)
    total_p1 += res
print(f"Total Part 1 Test: {total_p1}")

print("\nPart 2 Test:")
total_p2 = 0
for m in test_machines:
    res = solve_part2(m)
    print(res)
    total_p2 += res
print(f"Total Part 2 Test: {total_p2}")


In [ ]:
print("Running on Input...")
total_p1 = 0
total_p2 = 0

for i, m in enumerate(machines):
    res1 = solve_part1(m)
    total_p1 += res1
    
    res2 = solve_part2(m)
    total_p2 += res2
    
    if i % 20 == 0:
        print(f"Processed {i+1}/{len(machines)}")

print(f"Part 1 Solution: {total_p1}")
print(f"Part 2 Solution: {total_p2}")


# Results

Part 1: 479
Part 2: 19574


# Day 10: Factory

## Problem Understanding
We are given a list of machines. Each machine has:
1. **Indicator Lights**: A target configuration of lights (on/off). Initially all off.
2. **Buttons**: Each button toggles a specific set of lights.
3. **Joltage Requirements**: A target configuration of counters (integers). Initially all 0. Each button increments a specific set of counters.

We need to find the minimum number of button presses to reach the target state.

## Part 1: Indicator Lights
- The state is binary (on/off).
- Pressing a button twice is equivalent to not pressing it (modulo 2).
- We want to find a combination of button presses (0 or 1 for each button) that results in the target light configuration.
- We want to minimize the total number of presses.
- Since the number of buttons is small (up to ~12), we can iterate through all $2^N$ combinations of button presses.

## Part 2: Joltage Requirements
- The state is integer counts.
- Pressing a button adds to the counters.
- We want to find non-negative integer counts for each button press such that the total added to each counter equals the target joltage.
- We want to minimize the total number of presses.
- This is an Integer Linear Programming (ILP) problem.
- We can use `scipy.optimize.milp` to solve this efficiently.
- The constraints are: $A x = b$, $x \ge 0$, $x \in \mathbb{Z}$.
- The objective is to minimize $\sum x_i$.

In [ ]:
import re
import numpy as np
from itertools import product

def parse_line(line):
    # Extract lights part
    lights_match = re.search(r'\[([.#]+)\]', line)
    lights_str = lights_match.group(1)
    target_lights = [1 if c == '#' else 0 for c in lights_str]
    
    # Extract buttons
    buttons = []
    button_matches = re.findall(r'\(([\d,]+)\)', line)
    for b in button_matches:
        indices = [int(x) for x in b.split(',')]
        buttons.append(indices)
        
    # Extract joltage
    joltage_match = re.search(r'\{([\d,]+)\}', line)
    joltage_str = joltage_match.group(1)
    target_joltage = [int(x) for x in joltage_str.split(',')]
    
    return {
        'target_lights': target_lights,
        'buttons': buttons,
        'target_joltage': target_joltage
    }

def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return [parse_line(line.strip()) for line in lines if line.strip()]

machines = read_input('input.txt')
print(f"Parsed {len(machines)} machines.")
print(machines[0])

## Input Parsing
We parse the input file using regular expressions to extract the light patterns, button configurations, and joltage requirements.

In [ ]:
def solve_part1(machines):
    total_presses = 0
    
    for i, machine in enumerate(machines):
        target = np.array(machine['target_lights'])
        buttons = machine['buttons']
        num_buttons = len(buttons)
        num_lights = len(target)
        
        min_presses = float('inf')
        found = False
        
        # Iterate all 2^N combinations
        for presses in product([0, 1], repeat=num_buttons):
            current_state = np.zeros(num_lights, dtype=int)
            current_press_count = sum(presses)
            
            if current_press_count >= min_presses:
                continue
                
            for b_idx, press in enumerate(presses):
                if press:
                    for light_idx in buttons[b_idx]:
                        if light_idx < num_lights:
                            current_state[light_idx] ^= 1
            
            if np.array_equal(current_state, target):
                if current_press_count < min_presses:
                    min_presses = current_press_count
                    found = True
        
        if found:
            total_presses += min_presses
        else:
            print(f"Machine {i} has no solution for Part 1")
            
    return total_presses

part1_result = solve_part1(machines)
print(f"Part 1 Result: {part1_result}")

## Part 1 Solver
We iterate through all possible subsets of buttons (press or not press) and check if the resulting light pattern matches the target. We keep track of the minimum number of presses found.

In [ ]:
from scipy.optimize import milp, LinearConstraint, Bounds

def solve_part2_milp(machines):
    total_presses = 0
    
    for i, machine in enumerate(machines):
        target = np.array(machine['target_joltage'])
        buttons = machine['buttons']
        num_buttons = len(buttons)
        num_joltage = len(target)
        
        # Construct A matrix
        A = np.zeros((num_joltage, num_buttons))
        for j, btn_indices in enumerate(buttons):
            for idx in btn_indices:
                if idx < num_joltage:
                    A[idx, j] = 1
                    
        # Objective: minimize sum of x_j
        c = np.ones(num_buttons)
        
        # Constraints: Ax = target
        # milp uses b_l <= Ax <= b_u
        constraints = LinearConstraint(A, target, target)
        
        # Bounds: x >= 0
        bounds = Bounds(lb=0, ub=np.inf)
        
        # Integrality: all variables are integers
        integrality = np.ones(num_buttons)
        
        res = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
        
        if res.success:
            # Check if the solution is valid (sometimes milp returns slightly off values or fails to find integer feasible)
            # But with integrality=1 it should be integer.
            # Round to nearest integer just in case of float precision
            x = np.round(res.x).astype(int)
            
            # Verify solution
            if np.all(A @ x == target):
                total_presses += np.sum(x)
            else:
                print(f"Machine {i}: MILP returned invalid solution")
        else:
            print(f"Machine {i}: MILP failed to find solution")
            
    return int(total_presses)

part2_result = solve_part2_milp(machines)
print(f"Part 2 Result: {part2_result}")

## Part 2 Solver
We use `scipy.optimize.milp` to solve the integer linear programming problem.
We construct the matrix $A$ where $A_{ij} = 1$ if button $j$ affects counter $i$, else 0.
We set the constraints $Ax = \text{target}$ and $x \ge 0$.
We minimize the sum of $x$.

In [ ]:
print(f"Part 1: {part1_result}")
print(f"Part 2: {part2_result}")

## Results
We run the solvers on the input data and print the results. We also verify with the test data.

In [ ]:
test_machines = read_input('test.txt')
print(f"Test Part 1: {solve_part1(test_machines)}")
print(f"Test Part 2: {solve_part2_milp(test_machines)}")